In [1]:
import tensorflow as tf
from utils import create_session_config
import tensorflow_probability as tp
import numpy as np
from models.utils import covariance,correlation_matrix

In [3]:
tf.reset_default_graph()
cor_coef=-1
corr = np.array([[1.0,cor_coef],[cor_coef,1.0]], dtype=np.float32)
std_dev = np.array([[1.0,0.0],[0,4.0]], dtype=np.float32)
true_cov = np.matmul(std_dev, np.matmul(corr,std_dev))
print("true_cov: %s"%true_cov)

data_gen = tp.distributions.MultivariateNormalFullCovariance(loc=tf.constant([0.0] * 2),
                                                              covariance_matrix=tf.constant(true_cov))
sample_op = data_gen.sample([100])

cov = covariance(None, 2, {})
cor = correlation_matrix(cov)
learnt_distr = tp.distributions.MultivariateNormalFullCovariance(loc=tf.constant([0.0] * 2),
                                                              covariance_matrix=cov)
ll = learnt_distr.log_prob(sample_op)
adam = tf.train.AdamOptimizer(learning_rate=0.001)

error = tf.negative(tf.reduce_mean(ll))
true_error = tf.negative(tf.reduce_mean(data_gen.log_prob(sample_op)))
train = adam.minimize(error)

with tf.Session(config=create_session_config()) as session:
    session.run(tf.global_variables_initializer())
    print("error:", session.run(error))
    print("prior cov: ",session.run(cov))
    print("prior cor: ",session.run(cor))
    print("true_error: ",session.run(true_error))
    for step in range(10000):  
        session.run(train)
        if step%1000 == 0:
            print("error:", session.run(error))
    print("error:", session.run(error))
    print("learnt cov: ",session.run(cov))
    print("learnt cor: ",session.run(cor))
    print("true_error: ",session.run(true_error))

true_cov: [[ 1. -4.]
 [-4. 16.]]
error: 22.337933
prior cov:  [[[ 1.8894675  -0.4840941 ]
  [-0.4840941   0.40970263]]]
prior cor:  [[[ 1.        -0.5502065]
  [-0.5502065  1.       ]]]
true_error:  -inf
error: 26.037037
error: 7.3700867
error: 3.3186688
error: 3.4437957
error: 2.4722965
error: -1.5060239
error: -1.6890128
error: -1.7775111
error: -1.6727431
error: -1.7563491
error: -1.8560578
learnt cov:  [[[ 0.581436  -2.3256965]
  [-2.3256965  9.302765 ]]]
learnt cor:  [[[ 1.         -0.99999094]
  [-0.999991    1.        ]]]
true_error:  -inf
